In [43]:
from selenium import webdriver
import pandas as pd

In [2]:
kilburn = (51.541882,-0.1979358)
grove = (51.4318969,0.0212344)

In [3]:
def get_rect(topleft, botright):
    coords = []
    coords.append(topleft)
    coords.append(botright)
    
    width = botright[1] - topleft[1]
    height = botright[0] - topleft[0]
       
    topright = [topleft[0], botright[1]]
    coords.append(topright)
    
    botleft = [botright[0], topleft[1]]
    coords.append(botleft)
    
    return coords

In [30]:
co = get_rect(kilburn, grove)
co

[(51.541882, -0.1979358),
 (51.4318969, 0.0212344),
 [51.541882, 0.0212344],
 [51.4318969, -0.1979358]]

In [104]:
def get_section(topleft, width, height):
    '''
    Function to return a set of 4 Lat/Long pairs from
    topleft and w, h
    `return` list of tuples
    '''
    ## get bottom right
    BR = (topleft[0] - height, topleft[1] + width) 
    
    ## get topright
    TR = (topleft[0], BR[1])
    
    ## bottom left
    BL = BR[0], topleft[1]
    
    return [topleft, BR, TR, BL]

In [105]:
def round_tuple(tup):
    for num in tup:
        return tuple(map(round, tup, (6,6)))

In [106]:
round_tuple((-1.345678,52.345678))

(-1.345678, 52.345678)

In [138]:
def main_coords(topleft, botright, x=3, y=3):
    
    coords = {'main': get_rect(topleft, botright),
             'sections':[]}
    
    total_width = botright[1] - topleft[1]
    total_height = topleft[0] - botright[0] 
    
    section_width = total_width / x
    section_height = total_height / y
    
    for i in range(y):
        for j in range(x):
            ## get the coords with topleft, w, height
            temp_tl = (topleft[0] - i*section_height, topleft[1] + j*section_width)
            section_coords = get_section(temp_tl, section_width, section_height)
            coords['sections'].append( list(map(round_tuple,section_coords)))
    
    return coords

In [22]:
driver = webdriver.Chrome()
driver.get('https://www.mapcustomizer.com/')

In [139]:
address = driver.find_element_by_id('_location')

In [140]:
data = main_coords(kilburn, grove, x=5, y=5)

In [137]:
# main rect
for pair in data['main']:
    st = f'{pair[0]}, {pair[1]}\n'
    address.send_keys(st)

In [144]:
plotted = []
for section in data['sections']:
    for pair in section:
        if not pair in plotted:
            plotted.append(pair)
            st = f'{pair[0]}, {pair[1]}\n'
            address.send_keys(st)

In [127]:
api = 'https://data.police.uk/api/crimes-street/violent-crime?poly={}&date=2019-01-01'

In [141]:
def coord_string(li):
    q = ''
    for pair in li:
        q += f'{pair[0]},{pair[1]}:'
    return q

In [142]:
coord_string(bl_sec)

'51.4685586,-0.1979358:51.4318969,-0.12487906666666666:51.4685586,-0.12487906666666666:51.4318969,-0.1979358:'

In [122]:
crimes = pd.read_json(api.format(coord_string(bl_sec)))

In [123]:
crimes.category.value_counts()['violent-crime']

195

In [146]:
def get_violent_count(area):
    year_total = 0
    area = coord_string(area)
    for month in range(1,13):
        api = f'https://data.police.uk/api/crimes-street/violent-crime?poly={area}&date=2019-{month}'
        df = pd.read_json(api)
        year_total += len(df)
#         return df.category.value_counts()['violent-crime']
    return year_total

In [145]:
for idx, area in enumerate(data['sections'], start=1):
    print(f'area {idx} had {get_violent_count(area)} violent crimes in january')

area 1 had 97 violent crimes in january
area 2 had 149 violent crimes in january
area 3 had 174 violent crimes in january
area 4 had 145 violent crimes in january
area 5 had 94 violent crimes in january
area 6 had 88 violent crimes in january
area 7 had 230 violent crimes in january
area 8 had 96 violent crimes in january
area 9 had 109 violent crimes in january
area 10 had 92 violent crimes in january
area 11 had 92 violent crimes in january
area 12 had 117 violent crimes in january
area 13 had 153 violent crimes in january
area 14 had 92 violent crimes in january
area 15 had 60 violent crimes in january
area 16 had 92 violent crimes in january
area 17 had 172 violent crimes in january
area 18 had 97 violent crimes in january
area 19 had 67 violent crimes in january
area 20 had 90 violent crimes in january
area 21 had 39 violent crimes in january
area 22 had 62 violent crimes in january
area 23 had 33 violent crimes in january
area 24 had 49 violent crimes in january
area 25 had 63 vi

In [129]:
# crime = pd.read_json(api.format(coord_string(get_rect(kilburn, grove))))

In [147]:
for idx, area in enumerate(data['sections'], start=1):
    print(f'area {idx} had {get_violent_count(area)} violent crimes in 2019')

area 1 had 1149 violent crimes in 2019
area 2 had 1726 violent crimes in 2019
area 3 had 2143 violent crimes in 2019
area 4 had 1895 violent crimes in 2019
area 5 had 1109 violent crimes in 2019
area 6 had 1203 violent crimes in 2019
area 7 had 2872 violent crimes in 2019
area 8 had 1425 violent crimes in 2019
area 9 had 1484 violent crimes in 2019
area 10 had 1066 violent crimes in 2019
area 11 had 1210 violent crimes in 2019
area 12 had 1445 violent crimes in 2019
area 13 had 1734 violent crimes in 2019
area 14 had 1103 violent crimes in 2019
area 15 had 843 violent crimes in 2019
area 16 had 1099 violent crimes in 2019
area 17 had 2034 violent crimes in 2019
area 18 had 1317 violent crimes in 2019
area 19 had 877 violent crimes in 2019
area 20 had 963 violent crimes in 2019
area 21 had 486 violent crimes in 2019
area 22 had 850 violent crimes in 2019
area 23 had 479 violent crimes in 2019
area 24 had 686 violent crimes in 2019
area 25 had 886 violent crimes in 2019
